In [1]:
import cv2
import numpy as np

In [2]:
face_net=cv2.dnn.readNetFromCaffe(
    'deploy.prototxt',
    'res10_300x300_ssd_iter_140000.caffemodel'
)

In [3]:
plate_cascade=cv2.CascadeClassifier("haarcascade_russian_plate_number.xml")


In [16]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    if not ret:
        break
    plate_det_frame=frame.copy()
    h,w=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(cv2.resize(frame,(300,300)),1.0,(300,300),(104.0,177.0,123.0))
    face_net.setInput(blob)
    dets=face_net.forward()

    for i in range(dets.shape[2]):
        conf=dets[0,0,i,2]
        if conf>0.5:
            box=dets[0,0,i,3:7] * np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype("int")

            startX,startY=max(0,startX),max(0,startY)
            endX,endY=min(w,endX),min(h,endY)

            if endY>startY and endX>startX:
                face_roi=frame[startY:endY,startX:endX]
                frame[startY:endY,startX:endX]=cv2.GaussianBlur(face_roi,(99,99),30)

    cv2.imshow("Privacy",frame)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()